In [22]:
import urllib.request
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/car/car.data'
urllib.request.urlretrieve(url, 'car.data')

import numpy as np
data = np.loadtxt('car.data', dtype='str', delimiter=',')
X, y = data[:,:-1], data[:,-1]
X.shape, y.shape



((1728, 6), (1728,))

In [23]:
combinacoes = 1
for i in range(X.shape[1]):
    valores = set(X[:,i])
    combinacoes *= len(valores)
    print(valores)
print(combinacoes)

{'vhigh', 'med', 'low', 'high'}
{'vhigh', 'med', 'low', 'high'}
{'5more', '4', '2', '3'}
{'more', '4', '2'}
{'big', 'med', 'small'}
{'high', 'med', 'low'}
1728


In [19]:
from sklearn.metrics import accuracy_score
import numpy as np
from collections import Counter

def maisFrequente(y):
    #print(Counter(y.flat).most_common(1)[0][0])
    return Counter(y.flat).most_common(1)[0][0]

class ZeroR():
    def fit(self, X, y):
        self.resposta = maisFrequente(y)
    def predict(self, X):
        y = np.empty((X.shape[0]), dtype='<U5')
        y[:] = self.resposta
        return y

modelo = ZeroR()
modelo.fit(X,y)
ypred = modelo.predict(X)
accuracy_score(y, ypred), (ypred == y)

unacc


(0.7002314814814815, array([ True,  True,  True, ...,  True, False, False]))

In [13]:
def impureza(y): #Gini
    labels = list(set(y))
    labels.sort()
    probabilidades = np.zeros((len(labels),))
    for i,k in enumerate(labels):
        probabilidades[i] = sum(y==k)/len(y)
    result = 1 - sum(probabilidades**2)
    return result

ytmp = y[:]
valor = impureza(ytmp)
valor

0.457283763074417

In [24]:
def impurezaValor(x, y, valor):
    iguais = x==valor
    impurezaIguais = impureza(y[iguais])
    propIguais = sum(iguais)/len(y)
    impurezaDiferentes = impureza(y[~iguais])
    propDiferentes = sum(~iguais)/len(y)
    impurezaTotal = propIguais*impurezaIguais + propDiferentes*impurezaDiferentes
    print(impurezaIguais)
    print(impurezaDiferentes)
    print(impurezaTotal)
    return impurezaTotal
impurezaValor(X[:,0],y,'vhigh')

0.2777777777777777
0.5065360177564395
0.44934645776177407


0.44934645776177407

In [11]:
def impurezaMinima(X, y):
    impurezas = []
    caracteristicasValores = []
    for i in range(X.shape[1]):
        valores = sorted(list(set(X[:,i])))
        for valor in valores:
            caracteristicasValores.append([i,valor])
            impurezasValorCaracteristica = impurezaValor(X[:,i], y, valor)
            impurezas.append(impurezasValorCaracteristica)
    caracteristicasValores = np.array(caracteristicasValores)
    impurezas = np.array(impurezas)
    menorimpureza = np.argmin(impurezas)
    caracteristica, valor = caracteristicasValores[menorimpureza]
    return impurezas[menorimpureza], int(caracteristica), valor
impurezaMinima(X, y)

(0.38615712609310704, 3, '2')

In [12]:
from sklearn.metrics import accuracy_score
import numpy as np
from collections import Counter
from sklearn.base import BaseEstimator, ClassifierMixin

def maisFrequente(y):
    return Counter(y.flat).most_common(1)[0][0]

class Arvore(BaseEstimator, ClassifierMixin):
    def fit(self, X, y):
        self.impureza,self.caracteristica,self.valor = impurezaMinima(X,y)
        iguais = X[:,self.caracteristica] == self.valor
        if sum(iguais)>0 and sum(~iguais)>0:
            self.iguais = Arvore()
            self.iguais.fit(X[iguais,:],y[iguais])
            self.diferentes = Arvore()
            self.diferentes.fit(X[~iguais,:],y[~iguais])
        else:
            self.resposta = maisFrequente(y)
    def predict(self, X):
        y = np.empty((X.shape[0]), dtype='<U5')
        if hasattr(self, 'resposta'):
            y[:] = self.resposta
        else:
            iguais = X[:,self.caracteristica] == self.valor
            y[iguais] = self.iguais.predict(X[iguais,:])
            y[~iguais] = self.diferentes.predict(X[~iguais,:])
        return y

modelo = Arvore()
modelo.fit(X,y)
ypred = modelo.predict(X)
accuracy_score(y, ypred), (ypred == y)

(1.0, array([ True,  True,  True, ...,  True,  True,  True]))

In [25]:
from sklearn.model_selection import cross_validate

scores = cross_validate(Arvore(), X, y)
scores['test_score'], np.mean(scores['test_score'])

0.36606354447180656
0.49334612538033495
0.45401930769007326
0.5813936042524006
0.3885052631578948
0.44880031623519323
0.4799634202103338
0.4442274671591958
0.4547000149057304
0.0
0.4852099523313571
0.44378102731319496
0.4830534291133145
0.4441980266715282
0.45316682072285225
0.4746580411721171
0.445794
0.45377233844265463
0.48604539855157747
0.4464199881023202
0.45665607459319196
0.345679012345679
0.48755721999278157
0.4542948905588733
0.38750181448686316
0.47696870748299314
0.45547593724079694
0.4585255198487713
0.4564572718759794
0.4569735855884347
0.48174933645019113
0.44831371988247926
0.4568056890686347
0.4897060231734175
0.4453076503807778
0.4566803160599344
0.0
0.5869090060931796
0.3792400451817724
0.5866102589075561
0.36232104782211394
0.4343792314125165
0.5867828036567279
0.36766001638159707
0.4388511390201888
0.5195101417495498
0.4232466181061395
0.4547307806870523
0.4791185749461372
0.4457174311584231
0.4567383584863519
0.36318075806939776
0.49975014069747137
0.4529094117787

(array([0.62716763, 0.73121387, 0.75144509, 0.75362319, 0.8057971 ]),
 0.7338493758900897)